# Reddit Bot Commenters <a class="tocSkip">
Identifies likely bot commenters on Reddit using Benford's Law. See [original blog post](https://diybigdata.net/2020/03/using-benfords-law-to-identify-bots-on-reddit/) for a discussion on this technique.

The core of this code is the `generateBenfordsLawAnalysis()` function, which takes a user event log data frame that must have a user ID column and a event timestamp column, and it returns the chi squared score of close each user's activity is to the ideal Benford's Law distribution. Scores closer to zero mean the user's activity more closely adheres to the ideal distribution. 

In [1]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.window import Window as W

import pandas as pd

pd.set_option('display.max_colwidth', -1)

spark = SparkSession\
        .builder\
        .appName("RedditBotCommenters")\
        .getOrCreate()

In [2]:
reddit_df = spark.read.parquet('qfs:///data/reddit/processed')

reddit_df.printSchema()

root
 |-- approved_by: string (nullable = true)
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- banned_by: string (nullable = true)
 |-- body: string (nullable = true)
 |-- body_html: string (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created: long (nullable = true)
 |-- created_utc: long (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- downs: long (nullable = true)
 |-- edited: string (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- mod_reports: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- name: string (nullable = true)
 |-- num_reports: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- removal_reason: string (nullable = true)
 |-- rep

`generateBenfordsLawAnalysis`

A function to perform Benford's Law analysis against a data frame of user activities in order to determine which user's activities best (or least) adhere to the Benford's Law distribution. The data frame is ostensibly a event log keyed by a user ID and has a timestamp for each event row. Only the user ID and timesamps columns are used for analysis.

### Arguments <a class="tocSkip">
* `df` - The data frame with the timestamped user activity to be analyzed
* `user_col` - a string identifying the name of the column of df that contains the user IDs
* `timestamp_col` - a string identifying the name of the column of df that contains the event timestamps. Must be `T.LongType()`.
* `event_threshold` - the minimum number of events a user must have for the Benford's Law analysis to performed on it. Defaults to 100.

### Returns <a class="tocSkip">
A dataframe with the following columns:
* `user_col` - The user IDs. The column name will be the same as the original dataframe.
* `frequency_count` - the number of events found for the user
* `chi_squared` - the chi squared score indicating how similar the user's activity is to the ideal Benford's Law distribution.
* `digit_share` - A list containing the relative share each first digit has among the user's activity. The list is ordered from digit 1 to digit 9.



In [3]:
from math import log10, sqrt

def _getUsersAndDigit(df, user_col, event_threshold):
    digits_df = (
        spark
        .createDataFrame(
            [[1], [2], [3], [4], [5], [6], [7], [8], [9]],
            schema=T.StructType([
                T.StructField(
                    "first_digit", 
                    T.IntegerType()
                )
            ])
        )
        .coalesce(1)
    )
    users_and_digits = (
        reddit_df
        .groupBy(user_col)
        .agg(F.count('*').alias('count'))
        .filter(F.col('count') > event_threshold )
        .select(user_col)
        .repartition(user_col)
        .crossJoin(digits_df)
    )
    return users_and_digits

def _generateFirstDigitShare(df, user_col, timestamp_col):
    user_event_window = W.partitionBy(user_col).orderBy(timestamp_col)
    user_cum_dist_window = W.partitionBy(user_col).orderBy('first_digit')
    
    event_time_delta = F.col(timestamp_col) - F.lag(F.col(timestamp_col)).over(user_event_window)

    first_digit_share = (
        df
        .select(
            user_col,
            timestamp_col,
            event_time_delta.alias('time_delta')
        )
        .filter(F.col('time_delta').isNotNull())
        .withColumn(
            'first_digit',
            F.substring(F.col('time_delta').cast(T.StringType()), 0, 1).cast(T.IntegerType())
        )
        .withColumn(
            'first_digit_cum_dist',
            F.cume_dist().over(user_cum_dist_window)
        )
        .groupBy(user_col, 'first_digit', 'first_digit_cum_dist')
        .agg(
            F.count(timestamp_col).alias('frequency_count')
        )
        .withColumn(
            'first_digit_share',
            F.col('first_digit_cum_dist') 
                - F.coalesce(
                    F.lag('first_digit_cum_dist').over(user_cum_dist_window), 
                    F.lit(0)
                )
        )
        .repartition(user_col)
    )
    return first_digit_share

def _expectedBenfordsShare():
    digits = [1, 2, 3, 4, 5, 6, 7, 8, 9]
    expected_share_list = [(d, log10(d+1)-log10(d)) for d in digits]

    expected_share_df = (
        spark
        .createDataFrame(
            expected_share_list,
            schema=T.StructType([
                T.StructField(
                    'first_digit', 
                    T.IntegerType()
                ),
                T.StructField(
                    'expected_share',
                    T.DoubleType()
                )
            ])
        )
        .coalesce(1)
    )
    
    return expected_share_df

def generateBenfordsLawAnalysis(df, user_col, timestamp_col, event_threshold = 100):
    user_digts_df = _getUsersAndDigit(df, user_col, event_threshold)
    first_digit_share_df = _generateFirstDigitShare(df, user_col, timestamp_col)
    expected_share_df = _expectedBenfordsShare()
    
    finalized_first_digit_share_df = (
        first_digit_share_df
        .join(
            user_digts_df,
            on=[user_col,'first_digit'],
            how='right'
        )
        .na.fill(0)
        .cache()
    )    
    user_benford_distances = (
        finalized_first_digit_share_df
        .join(
            F.broadcast(expected_share_df),
            on='first_digit',
            how='inner'
        )
        .withColumn(
            'chi_squared_addends',
            F.pow(
                (F.col('first_digit_share') - F.col('expected_share')),
                F.lit(2)
            ) / F.col('expected_share')
        )
        .orderBy(user_col, 'first_digit')
        .groupBy(user_col)
        .agg(
            F.sum('frequency_count').alias('frequency_count'),
            F.sum('chi_squared_addends').alias('chi_squared'),
            F.collect_list(F.col('first_digit_share')).alias('digit_share')
        )
    )
    return user_benford_distances 

In [4]:
new_df = generateBenfordsLawAnalysis(reddit_df, 'author', 'created_utc')

new_df.orderBy(F.col('chi_squared').desc()).limit(50).toPandas()

,author,frequency_count,chi_squared,digit_share
0,the_yaya,235,15.830935,"[0.029787234042553193, 0.017021276595744678, 0.0, 0.0, 0.008510638297872339, 0.004255319148936176, 0.0042553191489361625, 0.9276595744680851, 0.008510638297872353]"
1,rSGSemployee,172,15.513564,"[0.06395348837209303, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0058139534883720895, 0.9186046511627907, 0.011627906976744207]"
2,ContentForager,18149,13.605131,"[0.003140668907377817, 0.0013774863628850074, 0.0007162929087002039, 0.0017080830899774092, 0.001652983635462008, 0.9888148107333737, 0.0018733814535235593, 0.00033059672709245813, 0.0003856961816077753]"
3,hkottler,107,13.514282,"[0.04672897196261682, 0.02803738317757009, 0.0, 0.009345794392523366, 0.0, 0.0, 0.009345794392523366, 0.8598130841121495, 0.04672897196261683]"
4,linked_list,497,13.460676,"[0.008048289738430584, 0.002012072434607645, 0.004024144869215292, 0.0, 0.0, 0.9839034205231388, 0.0, 0.002012072434607659, 0.0]"
5,morganmikolas,485,13.450058,"[0.002061855670103093, 0.0, 0.0, 0.0, 0.008247422680412371, 0.9835051546391753, 0.006185567010309256, 0.0, 0.0]"
6,I-Love_You,703,12.977491,"[0.017069701280227598, 0.004267425320056896, 0.0, 0.0014224751066856355, 0.002844950213371264, 0.9672830725462305, 0.0014224751066855834, 0.0014224751066856944, 0.004267425320056861]"
7,acini,20053,12.688674,"[0.014609822986786338, 0.006930939915233109, 0.006980802792321116, 0.00473697332336076, 0.003939167289952632, 0.9572176514584891, 0.0024931438544003592, 0.0015457491897282738, 0.001446023435552224]"
8,DigitalWinter,408,12.561376,"[0.07352941176470588, 0.00735294117647059, 0.004901960784313722, 0.0, 0.004901960784313722, 0.017156862745098048, 0.02205882352941177, 0.8308823529411765, 0.039215686274509776]"
9,educated_caucasian,774,11.994562,"[0.08397932816537468, 0.009043927648578809, 0.031007751937984496, 0.0, 0.0012919896640826989, 0.09431524547803616, 0.006459948320413439, 0.007751937984496138, 0.7661498708010336]"


In [5]:

new_df.orderBy(F.col('chi_squared').asc()).limit(50).toPandas()

,author,frequency_count,chi_squared,digit_share
0,gerg6111,5893,0.000059,"[0.3004240882103478, 0.1747243426632739, 0.1255301102629347, 0.09754028837998308, 0.07905004240882096, 0.06751484308736222, 0.05733672603901607, 0.05224766751484311, 0.045292620865139965]"
1,Fiji_Artesian,10193,0.000109,"[0.3008927695477288, 0.17659177867163744, 0.12577258903168842, 0.09840086333758458, 0.07887766113999806, 0.06524085156479942, 0.05856960659275967, 0.049936230746590815, 0.04571764936721279]"
2,Roboticide,27676,0.000116,"[0.3038372597196127, 0.1766512501806619, 0.1254877872524932, 0.09622055210290503, 0.07768463650816593, 0.06583321289203636, 0.056800115623645064, 0.05184997832056648, 0.045635207399913336]"
3,ivosaurus,19255,0.000122,"[0.3038691249026227, 0.17330563490002598, 0.12542196831991692, 0.09592313684757203, 0.07935601142560378, 0.06782653856141263, 0.05681641132173454, 0.05136328226434694, 0.04611789145676448]"
4,chicofaraby,50468,0.000126,"[0.3021320440675279, 0.17833082349211382, 0.1265752556075137, 0.09584291035903936, 0.07846556233653013, 0.06657684077038917, 0.05708567805341991, 0.05017040500911474, 0.04482048030435126]"
5,elementality22,19313,0.000128,"[0.3006782995909491, 0.1740796354786931, 0.12499352767565891, 0.09853466576917103, 0.07828923523015585, 0.06601770827939724, 0.058509812043701204, 0.05172681613421004, 0.04717029979806353]"
6,ademnus,40213,0.000131,"[0.3009474548031731, 0.17638574590306622, 0.12376594633576199, 0.09529256708029743, 0.07855668564891949, 0.0673414070076841, 0.05809066719717504, 0.05209758038445278, 0.047521945639469876]"
7,evildead4075,10353,0.000151,"[0.3000096590360282, 0.17569786535303777, 0.12537428764609287, 0.096783541002608, 0.08210180623973728, 0.06548826427122567, 0.05747126436781613, 0.051289481309765295, 0.04578383077368875]"
8,ISwearImAGirl,15329,0.000159,"[0.30099810816100203, 0.1780938091199687, 0.1267532128645052, 0.09667949637941153, 0.07893535129493123, 0.06667101572183443, 0.058059886489660095, 0.049970643877617626, 0.04383847609106917]"
9,WeaponsGradeHumanity,38608,0.000162,"[0.30355616566085625, 0.1729648527545391, 0.12445284778160537, 0.09580667719961666, 0.07990364940816908, 0.06840374006060757, 0.05742184464762112, 0.05068766349814802, 0.046776658292108086]"
